# Data Preprocessing

## Imports

In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from dvc_helper import DvcHelper
from df_overview import DfOverview
from df_cleaner import DfCleaner
from df_outlier import DfOutlier
from vis_seaborn import VisSeaborn
from vis_plotly import VisPlotly

2021-07-27 13:04:11,037 — DfOverview — DEBUG — Loaded successfully!
2021-07-27 13:04:11,041 — DfCleaner — DEBUG — Loaded successfully!
2021-07-27 13:04:11,047 — DfOutlier — DEBUG — Loaded successfully!


In [4]:
sb = VisSeaborn()

## Load Data

In [5]:
dvc_helper = DvcHelper()

In [6]:
missing_values = ["n/a", "na", "undefined", "nan"]

df_store = dvc_helper.read_csv('../data/store.csv', missing_values)
df_train = dvc_helper.read_csv('../data/train.csv', missing_values)
df_test = dvc_helper.read_csv('../data/test.csv', missing_values)

2021-07-27 13:04:11,403 — DfHelper — DEBUG — file read as csv
2021-07-27 13:04:11,926 — DfHelper — DEBUG — file read as csv
2021-07-27 13:04:11,965 — DfHelper — DEBUG — file read as csv


## Dataset Description

### Data fields:

- `Id` - an Id that represents a(Store, Date) duple within the test set
- `Store` - a unique Id for each store
- `Sales` - the turnover for any given day(this is what you are predicting)
- `Customers` - the number of customers on a given day
- `Open` - an indicator for whether the store was open: 0 = closed, 1 = open
- `StateHoliday` - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- `SchoolHoliday` - indicates if the(Store, Date) was affected by the closure of public schools
- `StoreType` - differentiates between 4 different store models: a, b, c, d
- `Assortment` - describes an assortment level: a = basic, b = extra, c = extended. Read more about assortment here
- `CompetitionDistance` - distance in meters to the nearest competitor store
- `CompetitionOpenSince[Month / Year]` - gives the approximate year and month of the time the nearest competitor was opened
- `Promo` - indicates whether a store is running a promo on that day
- `Promo2` - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- `Promo2Since[Year / Week]` - describes the year and calendar week when the store started participating in Promo2
- `PromoInterval` - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


### Information about data

Store data

In [7]:
print(df_store.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB
None


We have total 1115 unique stores


Train data

In [8]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB
None


As we can see here, we have around 1 million datapoints. 

Test data

In [9]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB
None


We have around 41000 datapoints for testing

## Clean Data

In [10]:
cleaner = DfCleaner()

### Missing value

Let's start by getting detailed overview the store data

In [11]:
store_overview = DfOverview(df_store)
store_overview.getOverview()

,count,none_count,none_percentage,unique_value_count,unique_percentage,dtype
label,,,,,,
Store,1115,0,0.0%,1115,100.0%,int64
StoreType,1115,0,0.0%,4,0.36%,object
Assortment,1115,0,0.0%,3,0.27%,object
Promo2,1115,0,0.0%,2,0.18%,int64
CompetitionDistance,1112,3,0.27%,654,58.65%,float64
CompetitionOpenSinceMonth,761,354,31.75%,12,1.08%,float64
CompetitionOpenSinceYear,761,354,31.75%,23,2.06%,float64
Promo2SinceWeek,571,544,48.79%,24,2.15%,float64
Promo2SinceYear,571,544,48.79%,7,0.63%,float64


In the store dataset, out of the 10 columns 6 have null value. Lets look at them one by one and decide what to do based on the data. There are only 3 missing values in `CompetitionDistance` and it holds continues numeric value. Since they are small we can impute them with median.

In [12]:
df_store = cleaner.fill_with_median(df_store, ['CompetitionDistance'])

Let's look into what data of `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear`.

In [14]:
print(np.sort(df_store['CompetitionOpenSinceMonth'].unique()))
print(np.sort(df_store['CompetitionOpenSinceYear'].unique()))

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. nan]
[1900. 1961. 1990. 1994. 1995. 1998. 1999. 2000. 2001. 2002. 2003. 2004.
 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]


Columns `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear` hold the year and month when  a new compitition opened. Since there is `CompetitionDistance` for all columns 
time when new compitition is started. 
Here both `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear` having null value is meaning full. It means competition for a store has already been there before the store is established or there is no compitition. 

The other point is our training data is collected starting from 2013. Becouse of this we can
replace null values and compitition that started before 2013 by first month 2013. This mean 
there was compitition before we started collecting the data.

In [19]:
df_store['CompetitionOpenSinceMonth'] = df_store['CompetitionOpenSinceMonth'].fillna(1)
df_store['CompetitionOpenSinceYear'] = df_store['CompetitionOpenSinceYear'].fillna(2013)


Now we are left with three columns. `Promo2SinceWeek`, `Promo2SinceYear` and `PromoInterval`. All of this columns have a null percentage of 48.79 % . Let's look into what data they are holding.


In [18]:
print(np.sort(df_store['Promo2SinceWeek'].unique()))
print(np.sort(df_store['Promo2SinceYear'].unique()))
print(df_store['PromoInterval'].unique())

[ 1.  5.  6.  9. 10. 13. 14. 18. 22. 23. 26. 27. 28. 31. 35. 36. 37. 39.
 40. 44. 45. 48. 49. 50. nan]
[2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]
[nan 'Jan,Apr,Jul,Oct' 'Feb,May,Aug,Nov' 'Mar,Jun,Sept,Dec']


In [25]:
df_store[df_store['Promo2'] == 0][['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].head(5)

,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


 As we can see, this three columns dependent on Promo2, and the reason for mesing value is because Promo2 is equal to 0, thus we can replace these nulls with 0.

For `Promo2SinceWeek` and `Promo2SinceYear` we can replace them with 0. This will indicate there
was no promo 2. 

In [26]:
df_store['Promo2SinceWeek'] = df_store['Promo2SinceWeek'].fillna(0)
df_store['Promo2SinceYear'] = df_store['CompetitionOpenSinceYear'].fillna(0)


Entries in `PromotionInterval` have 4 entries. We will split those points into 4 columns

In [28]:
import calendar

In [60]:
def getMonth(months, index):
  if(months == None):
    return 0
  print(months)
  month = months.split(',')[index]
  return list(calendar.month_abbr).index(month)


In [52]:
getMonth('Feb,May,Aug,Nov', 0)


2

1

In [61]:
df_store['PromoInterval0'] = df_store.PromoInterval.apply((lambda x: getMonth(x, 0)))
df_store['PromoInterval0']

nan


AttributeError: 'float' object has no attribute 'split'

In [ ]:
store_overview = DfOverview(df_store)
store_overview.missing_value()

[0, 0, 0, 0, 0, 0, 0, 544, 544, 544]

Now lets look at training data

In [ ]:
train_overview = DfOverview(df_train.copy())
train_overview.getOverview()[:10]

,count,none_count,none_percentage,unique_value_count,unique_percentage,dtype
label,,,,,,
Store,1017209,0,0.0%,1115,0.11%,int64
DayOfWeek,1017209,0,0.0%,7,0.0%,int64
Date,1017209,0,0.0%,942,0.09%,object
Sales,1017209,0,0.0%,21734,2.14%,int64
Customers,1017209,0,0.0%,4086,0.4%,int64
Open,1017209,0,0.0%,2,0.0%,int64
Promo,1017209,0,0.0%,2,0.0%,int64
StateHoliday,1017209,0,0.0%,5,0.0%,object
SchoolHoliday,1017209,0,0.0%,2,0.0%,int64


The training data is perfect. No Null values. This can't be coincedence, I think this data is cleaned before.

In [ ]:
test_overview = DfOverview(df_test)
test_overview.getOverview()

,count,none_count,none_percentage,unique_value_count,unique_percentage,dtype
label,,,,,,
Id,41088,0,0.0%,41088,100.0%,int64
Store,41088,0,0.0%,856,2.08%,int64
DayOfWeek,41088,0,0.0%,7,0.02%,int64
Date,41088,0,0.0%,48,0.12%,object
Promo,41088,0,0.0%,2,0.0%,int64
StateHoliday,41088,0,0.0%,2,0.0%,object
SchoolHoliday,41088,0,0.0%,2,0.0%,int64
Open,41077,11,0.03%,2,0.0%,float64


This is also almost perfect. Only 11 entries in `Open` column are null. We can just drop those.

In [ ]:
df_test = cleaner.drop_rows(df_test, 'Open', None)

In [ ]:
test_overview = DfOverview(df_test)
test_overview.missing_value()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

## Outlier treating

As before we will start with the store data. Here the only column we should consider is `CompetitionDistance` because the others have fewer unique values.

In [ ]:
df_outliers = DfOutlier(df_store[['CompetitionDistance']].copy())
overview = df_outliers.getOverview()
overview

In [ ]:
sb.boxplot(df_store, ['CompetitionDistance'], 'Competition Distance', figsize=(6, 4))

Now lets look at the training data. Here we should consider `Sales` and `Customers`.

In [ ]:
df_outliers = DfOutlier(df_train[['Sales', 'Customers']].copy())
overview = df_outliers.getOverview()
overview

In [ ]:
sb.boxplot(df_train, ['Sales', 'Customers'], cols=2, figsize=(12, 4))

For the test data we can do nothing.